In [16]:
from google.cloud import aiplatform
from vertexai.language_models import TextEmbeddingModel
from vertexai.generative_models import GenerativeModel
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import faiss
import numpy as np

In [13]:
# Step 1: 初始化 Vertex AI 项目与区域
aiplatform.init(
    project="vertex-ai-test-465220",
    location="us-central1"  # Gemini 支持区域
)

In [55]:
# 2. 加载文档并分割
loader = TextLoader("AI_Agent_Frameworks_Research.md")
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(documents)
chunks = [doc.page_content for doc in docs]

In [56]:
# 3. 使用 Vertex AI 的 text embedding 生成向量
embedding_model = TextEmbeddingModel.from_pretrained("text-embedding-004")
embeddings = embedding_model.get_embeddings(chunks)
vectors = np.array([e.values for e in embeddings]).astype("float32")

In [57]:
# ========== 4. 构建 FAISS 索引 ==========
dimension = vectors.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(vectors)

In [58]:
# ========== 5. 用户查询 ==========
question = "What are the key components of AutoGen and how is it used in multi-agent coordination?"
query_embedding = embedding_model.get_embeddings([question])[0].values
query_vector = np.array(query_embedding).astype("float32").reshape(1, -1)

In [59]:
# ========== 6. 检索最相关文本块 ==========
D, I = index.search(query_vector, k=3)
retrieved_chunks = [chunks[i] for i in I[0] if i < len(chunks)]

In [60]:
# ========== 7. 构建 Gemini 提示词（Prompt） ==========
context = "\n\n".join(retrieved_chunks)
prompt = f"""You are a helpful assistant. Use the following context to answer the question. Only answer with one sentence.

Context:
{context}

Question: {question}

Answer:"""

In [61]:
# ========== 8. 使用 Gemini 生成回答 ==========
model = GenerativeModel("gemini-2.5-pro")
response = model.generate_content(prompt)

/opt/conda/lib/python3.10/site-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


In [62]:
print("🔍 Gemini RAG Answer:\n")
print(response.text)

🔍 Gemini RAG Answer:

AutoGen is a Python-based framework with an asynchronous, event-driven architecture that coordinates multi-agent systems by enabling agents with clear roles to engage in conversations and automate complex workflows.


In [35]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_precision,
    context_recall
)
from ragas import evaluate
from datasets import Dataset

In [36]:
ground_truth = "AutoGen is for multi-agent coordination and planning."

In [63]:
# ========== 2. 构建 HuggingFace Dataset ==========
data = Dataset.from_dict({
    "question": [question],
    "answer": [response.text],
    "contexts": [[context]],
    "ground_truth": [ground_truth],
})

In [64]:
# ========== 3. 执行 RAGAS 评估 ==========
result = evaluate(
    data,
    metrics=[
        answer_relevancy,
        faithfulness,
        context_precision,
        context_recall
    ]
)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable